In [8]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame
import os

path=os.getcwd()

h2o.init()

# Import datasets into H2O
train = pd.read_csv(path + f'/h2o/train.csv', encoding='utf-8')
test = pd.read_csv(path + f'/h2o/test.csv', encoding='utf-8')

train_h2o = H2OFrame(train)
test_h2o = H2OFrame(test)
#train_h2o = h2o.import_file(path + f'/h2o/train.csv')
#test_h2o = h2o.import_file(path + f'/h2o/test.csv')

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,9 mins 22 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 5 days
H2O_cluster_name:,H2O_from_python_32170336_r3q1e9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.957 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# Set the target and predictors
y = 'ECLO'
x = train_h2o.columns
print(x)
x.remove(y)
delete = ['Unnamed: 0', '사망자수', '중상자수', '경상자수', '부상자수']

for i in delete:
    x.remove(i)


# For demonstration, assuming 'ID' column exists in the test dataset and needs to be removed
test_h2o = test_h2o.drop('ID', axis=1)


['Unnamed: 0', '요일', '기상상태', '노면상태', '사고유형', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '시', '구', '동', '도로형태1', '도로형태2', '년도', '월', '시간', 'holiday', '시간구분', '계절', '기상상태_레이블', '노면상태_레이블', '사고유형_레이블', '시_레이블', '구_레이블', '동_레이블', '도로형태1_레이블', '도로형태2_레이블', 'holiday_레이블', '시간구분_레이블', '계절_레이블']


H2OValueError: Column(s) selected to drop are not in original frame: 'ID'

In [12]:
# Initialize and run H2O AutoML
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=1000, stopping_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_h2o)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb)

AutoML progress: |█
15:00:14.518: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
model_id                                                 rmse      mse      mae     rmsle    mean_residual_deviance
GBM_grid_1_AutoML_1_20240226_150014_model_3           3.11545  9.70603  2.02406  0.434423                   9.70603
GBM_2_AutoML_1_20240226_150014                        3.11559  9.70688  2.02355  0.434436                   9.70688
GBM_5_AutoML_1_20240226_150014                        3.11572  9.70772  2.02451  0.434422                   9.70772
GBM_grid_1_AutoML_1_20240226_150014_model_2           3.11596  9.70919  2.02539  0.434454                   9.70919
GBM_3_AutoML_1_20240226_150014                        3.1175   9.71882  2.02512  0.4346                     9.71882
GBM_4_AutoML_1_20240226_150014                        3.12313  9.75396  2.02797  0.435327                   9.75396
DeepLearning_1_AutoML_1_202402

In [13]:
# Generate predictions
preds = aml.predict(test_h2o)
h2o_predictions = preds.as_data_frame()
print(h2o_predictions.head())

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\Users\32170336\AppData\Local\Programs\Python\Python312\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


    predict
0  4.013062
1  4.489311
2  3.843454
3  5.524245
4  5.204374
